In [1]:
import os
import pandas as pd
from pathlib import Path
import openpyxl
import tabula

In [3]:
download_path = "./Download"
file_path = (f"{download_path}/2017_1.xlsx")
df = pd.read_excel(file_path, engine='openpyxl')
df.tail()

,Počty podnikatelů dle občanství podnikajících v České republice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
161,Jemen,887,YE,YEM,10,1,0,2,0,3,1,0,0,1,3,1,1,0,23
162,Srbsko a Černá Hora,891,CS,SCG,259,54,16,15,10,24,11,5,1,2,21,8,15,7,448
163,Zambie,894,ZM,ZMB,1,2,0,0,0,0,0,0,0,0,0,0,0,0,3
164,Součet,SUM1,NaN,NaN,328403,257904,123176,108217,55388,131755,91623,106193,90211,85226,218492,106506,187649,108161,1998904
165,Z toho cizinci,SUM2,NaN,NaN,31712,9528,2852,5507,4934,7874,2598,2546,2160,1532,7097,2118,4124,1814,86396


In [20]:
df = df[3:]
df.head()

,Počty podnikatelů dle občanství podnikajících v České republice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
3,Občanství,Kód,Kód zn.,Kód zn.,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,Liberecký kraj,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet
4,Afghánistán,004,AF,AFG,16,7,0,0,1,6,2,0,0,1,13,4,2,0,52
5,Albánie,008,AL,ALB,13,8,4,1,1,3,1,2,1,1,2,1,1,4,43
6,Alžírsko,012,DZ,DZA,94,22,3,0,1,7,2,1,1,1,9,2,2,0,145
7,Angola,024,AO,AGO,9,2,2,0,0,3,2,1,1,0,6,3,2,2,33


In [100]:
# Automatizace převodu celé složky .XLSX na správné .CSV
# Otevře, převede na .csv, odebere 4 první řádky, 
# resetuje index, uloží jako .csv

download_path = "./Download"
export_path = "./Preprocessed_CSV"


folder_path = Path(download_path)

'''
TEST - vypíše všechny soubory v Download složce

files = list(folder_path.iterdir())
for x in files:
    print(x)
'''

processed = 0
all_dataframes = []

# Prochází všechny soubory ve složce
for file in folder_path.iterdir():
    if file.is_file() and not file.name.startswith('.'):  # Kontrola, zda jde o soubor

        # Definice nových názvů
        file_path = (f"{download_path}/{file.name}")  # Proměnná filepath
        newname = (file.name).replace(".xlsx", "")  # Odstranění .xlsx ze starého názvu
        csv_path = (f"{export_path}/{newname}.csv")  # Nový název pro uložení

        # Otevře excel v pandasu
        df = pd.read_excel(file_path, engine='openpyxl')  # Načíst excel jako pandas df

        # Stav
        processed += 1
        print(f"{processed} Soubor {csv_path} úspěšně převeden", end=" ")

        # Zahodíme 6 zbytečných řádků, poslední sloupec + resetujeme index
        df = df[3:].reset_index(drop=True) # Odstraní první 4 řádky
        df = df.iloc[:,:-1]  # Odstraní poslední sloupec
        df = df[:-2]  # Odstraní poslední 2 řádky
        df.columns = df.iloc[0]  # Nastaví 5. řádek (který je teď první po odstranění) jako názvy sloupců
        df = df[1:]  # Odstraní řádek použitý jako názvy sloupců
        df = df.set_index("Občanství")  # Nastaví sloupec "Občanství" jako index
        df = df.sort_values(by=df.columns[0], ascending=True)  # Utřídí názvy řad abecedně


        # Definice proměnných q a rok
        q = newname[-1]
        rok = newname[-6:-2]

        # Přidání suffixu do názvu sloupců
        for i, col in enumerate(df.columns):
            if i >= 4:  # Začíná na 5. sloupci
                df.rename(columns={col: f"{col} {rok} Q{q}"}, inplace=True)

        # Reset
        #df.reset_index(drop=True, inplace=True)  #  Pro jistotu ještě jednou index reset

        # Append the processed DataFrame to the list for merging
        all_dataframes.append(df)

        # Uloží jako CSV
        df.to_csv(csv_path, index=True)
        shape = df.shape
        print(f"+ uložen + přidán df do listu. Shape: {shape}", end="\n")


# Část s mergem v další složce


1 Soubor ./Preprocessed_CSV/2024_3.csv úspěšně převeden + uložen + přidán df do listu. Shape: (252, 17)
2 Soubor ./Preprocessed_CSV/2019_1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
3 Soubor ./Preprocessed_CSV/2016_4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
4 Soubor ./Preprocessed_CSV/2017_1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
5 Soubor ./Preprocessed_CSV/2024_2.csv úspěšně převeden + uložen + přidán df do listu. Shape: (252, 17)
6 Soubor ./Preprocessed_CSV/2018_4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
7 Soubor ./Preprocessed_CSV/2018_3.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
8 Soubor ./Preprocessed_CSV/2018_2.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
9 Soubor ./Preprocessed_CSV/2017_4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
10 Soubor ./Preprocessed_CSV/2018_1.csv úspěšně převeden + ulože

In [98]:
df.head()

,Kód,Kód zn.,Kód zn.,Hlavní město Praha,Středočeský kraj 2017 Q3,Jihočeský kraj 2017 Q3,Plzeňský kraj 2017 Q3,Karlovarský kraj 2017 Q3,Ústecký kraj 2017 Q3,Liberecký kraj 2017 Q3,Královéhradecký kraj 2017 Q3,Pardubický kraj 2017 Q3,Kraj Vysočina 2017 Q3,Jihomoravský kraj 2017 Q3,Olomoucký kraj 2017 Q3,Moravskoslezský kraj 2017 Q3,Zlínský kraj 2017 Q3
Občanství,,,,,,,,,,,,,,,,,
Afghánistán,004,AF,AFG,16,7,0,0,2,7,2,0,0,1,12,3,2,0
Albánie,008,AL,ALB,14,7,4,1,1,3,1,2,1,1,2,1,1,4
Alžírsko,012,DZ,DZA,92,19,4,0,1,7,2,1,1,1,8,3,2,1
Angola,024,AO,AGO,10,2,2,0,0,2,3,1,1,0,6,3,1,2
Ázerbájdžán,031,AZ,AZE,58,4,1,1,4,4,0,1,0,0,2,3,0,0


In [89]:
for i in df.columns:
    print(i)

Občanství
Kód
Kód zn.
Kód zn.
Hlavní město Praha 2017 Q3
Středočeský kraj 2017 Q3
Jihočeský kraj 2017 Q3
Plzeňský kraj 2017 Q3
Karlovarský kraj 2017 Q3
Ústecký kraj 2017 Q3
Liberecký kraj 2017 Q3
Královéhradecký kraj 2017 Q3
Pardubický kraj 2017 Q3
Kraj Vysočina 2017 Q3
Jihomoravský kraj 2017 Q3
Olomoucký kraj 2017 Q3
Moravskoslezský kraj 2017 Q3
Zlínský kraj 2017 Q3


In [68]:
'''
# Merge all DataFrames into one large DataFrame
merged_df = pd.concat(all_dataframes, axis=0, join="outer")  # Full outer join on indexes and columns
merged_df.reset_index(drop=True, inplace=True)  # Reset the index after merging
'''

# Loop through the list of DataFrames and merge them progressively
merged_df = all_dataframes[0]  # Start with the first DataFrame

# Merge progressively
for df in all_dataframes[1:]:
    # Merge only the columns that start from the 5th column onwards
    merged_df = merged_df.merge(df, how="outer", left_index=True, right_index=True)

# Optional: Save the merged DataFrame to a new CSV file
merged_csv_path = f"{export_path}/merged_dataset_2.csv"
merged_df.to_csv(merged_csv_path, index=False)
print(f"\nVšechny soubory sloučeny do: {merged_csv_path}")

print(f"Ve složce je {processed} souborů.")
#print(f"Toto jsou jejich jména: {filenames}")

MergeError: Passing 'suffixes' which cause duplicate columns {'Občanství_x', 'Kód zn._x', 'Kód_x'} is not allowed.

In [45]:
merged_df.shape

(252, 228)

In [43]:
for x in merged_df.isna().sum().sort_values():

0
Občanství                        0
Olomoucký kraj 2024 Q1           0
Jihomoravský kraj 2024 Q1        0
Kraj Vysočina 2024 Q1            0
Pardubický kraj 2024 Q1          0
                                ..
Plzeňský kraj 2018 Q1           92
Jihočeský kraj 2018 Q1          92
Středočeský kraj 2018 Q1        92
Moravskoslezský kraj 2017 Q1    92
Liberecký kraj 2020 Q1          92
Length: 228, dtype: int64

In [44]:
unique_rows = merged_df.index.nunique()
duplicated_rows = merged_df.index[merged_df.index.duplicated()].unique()

print(f"Number of unique row indices: {unique_rows}")
print(f"Duplicated row indices: {duplicated_rows}")

Number of unique row indices: 252
Duplicated row indices: Index([], dtype='int64')


In [256]:
# Definice proměnných pro další zpracování tabulek

csv_file = "./Download_CSV/2016_2.csv"

suffix = csv_file.strip(".csv")
suffix = suffix[-1]

rok = csv_file.strip(".csv")
rok = rok[-6:-2]

In [257]:
# Otevřeme v pandasu jeden soubor a zkusíme řádky pojmenovat s příponou Q + číslo podle názvu souboru

df = pd.read_csv(csv_path)  # Read without headers
df["Rok"] = rok
df["Q"] = suffix
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,Rok,Q
0,Občanství,Kód,Kód zn.,Kód zn.,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,...,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet,2016,2
1,Afghánistán,004,AF,AFG,16,7,0,0,2,7,...,0,0,1,12,3,2,0,52,2016,2
2,Albánie,008,AL,ALB,14,7,4,1,1,3,...,2,1,1,2,1,1,4,43,2016,2
3,Alžírsko,012,DZ,DZA,92,19,4,0,1,7,...,1,1,1,8,3,2,1,142,2016,2
4,Angola,024,AO,AGO,10,2,2,0,0,2,...,1,1,0,6,3,1,2,33,2016,2


In [275]:
# Automat - přidá dva sloupce "rok" a "Q" jako kvartál

folder_path = Path("./Download_CSV")

for file in folder_path.iterdir():
    if file.is_file() and not file.name.startswith('.'):
        csv_file = file.name
        csv_path = (f"{folder_path}/{csv_file}")

        # Definice proměnných q, rok
        q = csv_file.strip(".csv")
        q = suffix[-1]

        rok = csv_file.strip(".csv")
        rok = rok[-6:-2]

        df = pd.read_csv(csv_path)
        df["Rok"] = rok
        df["Q"] = q

        df.to_csv(csv_path, index=False)

        print(f"Soubor {csv_file} byl změněn", end="\n")


Soubor 2016_2.csv byl změněn


In [276]:
df.head(5)

,Občanství,Kód,Kód zn.,Kód zn..1,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,...,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet,Rok,Q
0,Afghánistán,4,AF,AFG,15,6,0,0,1,6,...,0,0,1,12,4,2,0,48,2016,2
1,Albánie,8,AL,ALB,10,8,4,1,1,2,...,2,1,0,2,2,1,3,38,2016,2
2,Alžírsko,12,DZ,DZA,99,21,3,0,1,9,...,1,1,0,8,2,3,0,150,2016,2
3,Angola,24,AO,AGO,9,2,2,0,0,3,...,1,1,0,6,3,4,2,35,2016,2
4,Ázerbájdžán,31,AZ,AZE,37,5,0,2,4,4,...,1,0,0,2,4,0,0,59,2016,2


In [51]:
df.shape
#df.columns

(162, 18)

In [52]:
df.columns

Index(['Občanství', 'Kód', 'Kód zn.', 'Kód zn.', 'Hlavní město Praha 2017 Q3',
       'Středočeský kraj 2017 Q3', 'Jihočeský kraj 2017 Q3',
       'Plzeňský kraj 2017 Q3', 'Karlovarský kraj 2017 Q3',
       'Ústecký kraj 2017 Q3', 'Liberecký kraj 2017 Q3',
       'Královéhradecký kraj 2017 Q3', 'Pardubický kraj 2017 Q3',
       'Kraj Vysočina 2017 Q3', 'Jihomoravský kraj 2017 Q3',
       'Olomoucký kraj 2017 Q3', 'Moravskoslezský kraj 2017 Q3',
       'Zlínský kraj 2017 Q3'],
      dtype='object', name=0)

In [53]:
for x in df["Občanství"].values:
    print(x)

Afghánistán
Alandy
Albánie
Alžírsko
Angola
Argentina
Arménie
Austrálie
Bangladéš
Belgie
Benin
Bolívie
Bosna a Hercegovina
Botswana
Brazílie
Britské Panenské ostrovy
Bulharsko
Burkina Faso
Bělorusko
Chile
Chorvatsko
Curacao
Demokratická republika Kongo
Dominikánská republika
Dánsko
Egypt
Ekvádor
Eritrea
Estonsko
Etiopie
Filipíny
Finsko
Francie
Gambie
Ghana
Gruzie
Guatemala
Guernsey
Guinea
Guinea-Bissau
Guyana
Honduras
Hongkong
Indie
Indonésie
Irsko
Irák
Island
Itálie
Izrael
Jamajka
Japonsko
Jemen
Jižní Afrika
Jordánsko
Kambodža
Kamerun
Kanada
Katar
Kazachstán
Keňa
Kolumbie
Konžská republika
Korejská lidově demokratická republika
Korejská republika
Kosovo
Kostarika
Kuba
Kuvajt
Kypr
Kyrgyzstán
Laos
Libanon
Libye
Libérie
Litva
Lotyšsko
Lucembursko
Madagaskar
Makedonie
Malajsie
Mali
Malta
Maroko
Mauricius
Mauritánie
Maďarsko
Mexiko
Moldavsko
Monako
Mongolsko
Myanmar
Namibie
Nepál
Niger
Nigérie
Nizozemsko
Nizozemské Antily
Norsko
Nový Zéland
Německo
Palestina
Panama
Paraguay
Peru
Pobřeží slo

In [57]:
merged = pd.read_csv("Preprocessed_CSV/merged_dataset.csv")
merged.shape

(252, 228)

In [62]:
merged["Občanství"].values

array(['Afghánistán', 'Alandy', 'Albánie', 'Alžírsko', 'Americká Samoa',
       'Americké Panenské ostrovy', 'Andorra', 'Angola', 'Anguilla',
       'Antarktida', 'Antigua a Barbuda', 'Argentina', 'Arménie', 'Aruba',
       'Austrálie', 'Bahamy', 'Bahrajn', 'Bangladéš', 'Barbados',
       'Belgie', 'Belize', 'Benin', 'Bermudy', 'Bhútán', 'Bolívie',
       'Bonaire, Svatý Eustach a Saba', 'Bosna a Hercegovina', 'Botswana',
       'Bouvetův ostrov', 'Brazílie', 'Britské Panenské ostrovy',
       'Britské indickooceánské území', 'Brunej', 'Bulharsko',
       'Burkina Faso', 'Burundi', 'Bělorusko', 'Chile', 'Chorvatsko',
       'Cookovy ostrovy', 'Curaçao', 'Dominika', 'Dominikánská republika',
       'Dánsko', 'Džibutsko', 'Egypt', 'Ekvádor', 'Eritrea', 'Estonsko',
       'Etiopie', 'Faerské ostrovy', 'Falklandy (Malvíny)', 'Fidži',
       'Filipíny', 'Finsko', 'Francie', 'Francouzská Guyana',
       'Francouzská Polynésie', 'Francouzská jižní a antarktická území',
       'Gabon', 'Gambie

In [273]:
''''
JEDNORÁZOVÉ SPRAVENÍ SOUBORU 2016_2.csv

file_path = 'Down/2016_2.csv'  # Replace with your file path
df = pd.read_csv(file_path, sep=';')

# Save the file with commas as the delimiter
output_path = 'fixed_file.csv'  # Replace with your desired output path
df.to_csv(output_path, index=False, sep=',')
'''

In [ ]:
'''
------------------------------ THE END ---------------------------------
'''